In [1]:
import pandas as pd
import re
import time
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from nltk.corpus import stopwords
from selenium import webdriver
from textacy.preprocess import preprocess_text
import nltk

from bs4 import BeautifulSoup as bs

In [2]:
start = time.time()

wd = webdriver.Chrome()

b_summary = []
b_detail = []
b_date = []
b_summary_clean = []
b_detail_clean = []

for i in range(1,3):
    print 'scrapping page : ', i
    url = 'https://thechosenglutton.wordpress.com/page/' + str(i) + '/'   

    try:
        wd.get(url)
    except:
        continue
    response = wd.page_source
    soup = bs(response, 'html.parser')
    
    try:
        blogs = soup.find_all('h1', {'class' : 'entry-title'})
    except:
        continue
        
    for summary in blogs:
        try:
            href = summary.find('a').get('href')
            print 'href : ', href
        except:
            continue            
        
        try:
            wd.get(href)
        except:
            continue
        
        r = wd.page_source
        s = bs(r, 'html.parser')

        # find date
        try:
            date = s.find('time', {'class' : 'entry-date'}).text
        except:
            continue
            
        # find detail
        try:
            y = s.find('div', {'class' : "entry-content"}).find_all('p')
        except:
            continue
        doc = []
        for p in y:
            if p.text:
                doc.append(p.text)
        d = ' '.join(doc)

        b_summary.append(summary.text)
        b_detail.append(d)
        b_date.append(date)
        
        # clean summary and detail 
        clean_summary = preprocess_text(summary.text, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        clean_detail = preprocess_text(d, fix_unicode=True, lowercase=True, transliterate=True, 
                                                no_numbers=True, no_urls=True, no_emails=True, no_phone_numbers=True, 
                                                no_currency_symbols=True, no_punct=True, no_accents=True)
        
        b_summary_clean.append(clean_summary)
        b_detail_clean.append(clean_detail)
        
        print 'cleaned detail snippet : ', clean_detail[:75]
        
end = time.time()
exe_time = (end - start)/60
print 'Time taken :',(exe_time),' minutes'

print len(b_date)

scrapping page :  1
href :  https://thechosenglutton.wordpress.com/2017/10/29/level33craftbeerinspiredmenu/
cleaned detail snippet :  level33 level33 worlds highest urban craftbrewery when level33 prides itsel
href :  https://thechosenglutton.wordpress.com/2017/10/22/soi47/
cleaned detail snippet :  soi number toa payoh soi number sits on the ground floor of a hdb block in 
href :  https://thechosenglutton.wordpress.com/2017/08/27/ramenchampioncentral/
cleaned detail snippet :  ramen champion central clarke quay i cant believe it has been more than six
href :  https://thechosenglutton.wordpress.com/2017/08/24/thelimetruck/
cleaned detail snippet :  celebrity chef daniel shemtob flown in from halfway across the world in los
href :  https://thechosenglutton.wordpress.com/2017/07/01/maisonikkokubar/
cleaned detail snippet :  maison ikkoku bar 6th birthday special king prawn udon king prawn udon usdn
href :  https://thechosenglutton.wordpress.com/2017/06/14/marriottdurianhighteabuffet/
cle

In [3]:
# remove stop words from cleaned summary and cleaned detail and lemmatize them

stoplist = set(stopwords.words('english') + \
               ['number',"singapore", "food", "im", "street", 
                'porn','watch', 'video', 'centre', '...',
                'eat', 'day', 'time', 'cdataadsbygoogle', 'windowadsbygoogle',
                'wa', 'ha', 'come', 'place', 'dish', 'bring', 'think', 'quite','located',
                'month', 'went', 'probably', 'am', 'pm', 'say', 'said','including','year','item',
                'youre', 'sure', 'dont', 'came','really', 'got', 'thing', 'address', 'photo',
                'credit', 'opening', 'hour'] \
                 + list(ENGLISH_STOP_WORDS))

In [4]:
start = time.time()

summary_tokens = []
for item in b_summary_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    summary_tokens.append(t)
    
detail_tokens = []
for item in b_detail_clean:  
    try:
        Tokens = nltk.word_tokenize(item)
        t = [tok for tok in Tokens if tok not in stoplist]
    except:
        continue
    detail_tokens.append(t)   

    
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'
    
print len(summary_tokens)
print len(detail_tokens)

Time taken : 0.148549079895  seconds
14
14


In [5]:
start = time.time()

summary_lem =[]
wnl = WordNetLemmatizer()
for item in summary_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    summary_lem.append(t) 
    
detail_lem =[]
wnl = WordNetLemmatizer()
for item in detail_tokens:
    t = " ".join([wnl.lemmatize(i) for i in item])
    detail_lem.append(t)     
   
end = time.time()
exe_time = end - start
print 'Time taken :',(exe_time),' seconds'

print len(summary_lem)
print len(detail_lem)

Time taken : 4.21898794174  seconds
14
14


In [6]:
# create final list to fit in dataframe
flist = []
for i in range(len(b_summary)):
    f = [b_summary[i], b_detail[i], b_date[i], summary_lem[i], detail_lem[i]]
    flist.append(f)

In [7]:
df = pd.DataFrame(flist, columns = ['Title', 'Details', 'Date', 'Processed Summary', 'Processed Detail'])  
df.tail()

,Title,Details,Date,Processed Summary,Processed Detail
9,The Carvery @ Park Hotel Alexandra: Weekend Wo...,The Carvery @ Park Hotel Alexandra The Carvery...,"May 1, 2017",carvery park hotel alexandra weekend woodfired...,carvery park hotel alexandra carvery park hote...
10,Coastes: A Beachside Dining Experience,Coastes @ Siloso Beach Coastes @ Siloso Beach ...,"April 11, 2017",coastes beachside dining experience,coastes siloso beach coastes siloso beach coas...
11,Flying Monkey: Promising Indian Tapas & Indian...,Flying Monkey @ Bussorah Street At the heart o...,"April 9, 2017",flying monkey promising indian tapa indianinsp...,flying monkey bussorah heart kampong glam buss...
12,Plaza Brasserie @ PARKROYAL on Beach Road: Ret...,Curry Crab Meat Kueh Pie Tee Stir-fried Green ...,"April 7, 2017",plaza brasserie parkroyal beach road return cr...,curry crab meat kueh pie tee stirfried green m...
13,J65 @ Hotel Jen Tanglin: Different Day of the ...,"J65 @ Hotel Jen Tanglin Earlier this month, J6...","March 31, 2017",j65 hotel jen tanglin different week different...,j65 hotel jen tanglin earlier j65 hotel jen ta...


In [8]:
len(df)

14

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
len(df)

14

In [11]:
x = df.iloc[3,1]
print x

Celebrity Chef Daniel Shemtob Flown in from halfway across the world in Los Angeles is The Lime Truck, an award-winning food truck. It is a concept that has yet to find its way through or around the local legislation (I am guessing due to licensing, hygiene and cleanliness issues), but essentially a mobile eatery serving up typically handcrafted cuisines to where the demand is. But that has not stopped the winner of Food Network’s “The Great Food Truck Race” and Yahoo!’s “Top 10 Trucks in America” from setting up shop here (literally, since mobile food trucks ain’t a thing yet) in the little red dot, because let’s be frank, nothing can prevent our food culture from flourishing into an even bigger melting pot of vibrant cuisines and concepts. Parking its “truck” at Pasarbella at Suntec City, one can already imagine it to be based in a casual and high-spirited setting, not too far off from how it is like back at their hometown. Interestingly, it is the Celebrity Chef Daniel Shemtob’s fir

In [12]:
# pickle dataframe
df.to_pickle('chosenglutton.pickle')

In [13]:
wd.quit()